In [1]:
import os
import numpy as np
import pandas as pd
import warnings

from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/drive")

os.chdir("drive/MyDrive/AI competition/Round 1")
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['2022-08-06 Optimize ET.ipynb',
 'competition_data',
 '__pycache__',
 'html',
 'submission',
 'logs.log',
 'playground.ipynb',
 'AutoML Baseline.ipynb',
 'Evaluator Module.ipynb',
 'evaluator.py',
 '2022-08-04 feature EDA.ipynb',
 "2022-08-05 LGB_train(hyeonbin's parameter tuning).ipynb",
 'model compare.ipynb',
 'Optuna Optimization.ipynb',
 '2022-08-06 Regressor model compare.ipynb']

In [2]:
from evaluator import Evaluator, Model

train_df = pd.read_csv('competition_data/train.csv')
test_df = pd.read_csv("competition_data/test.csv")
submission_df = pd.read_csv("competition_data/sample_submission.csv")
train_df.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0


### baseline

In [3]:
Evaluator(
    **Model(train_df, "et", "rgr").get_model()
).run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.777743,0.793085,0.817372,0.805045,0.868221,0.292085,NaN
2,0.784308,0.784441,0.833046,0.808013,0.873223,0.293501,NaN
3,0.775242,0.782516,0.825183,0.803283,0.864512,0.300653,NaN
4,0.778056,0.803314,0.814634,0.808934,0.864605,0.295921,NaN
mean,0.778837,0.790839,0.822559,0.806319,0.867640,0.295540,1.0


### optimize

In [4]:
model = Model(train_df, "et", "rgr")

In [5]:
initial_params = (
    ("n_estimators", "static", 100),
    ("max_depth", "int", (10, 100)),
    ("min_samples_split", "int", (2, 10)),
    ("min_weight_fraction_leaf", "float", (0.0, 0.2)),
    ("max_features", "float", (0.7, 1.0)),
    # ("max_leaf_nodes", "static", None),
)
model.optimize(initial_params)

[I 2022-08-06 07:40:59,440] A new study created in memory with name: no-name-70ddf660-00a1-4979-83f6-5e7c28896e20
[I 2022-08-06 07:41:04,836] Trial 0 finished with value: 0.7721207288210089 and parameters: {'max_depth': 66, 'min_samples_split': 9, 'min_weight_fraction_leaf': 0.16278616336581087, 'max_features': 0.7869377009654474}. Best is trial 0 with value: 0.7721207288210089.
[I 2022-08-06 07:41:13,635] Trial 1 finished with value: 0.7709018435726687 and parameters: {'max_depth': 98, 'min_samples_split': 7, 'min_weight_fraction_leaf': 0.09535417157181768, 'max_features': 0.995863996401672}. Best is trial 0 with value: 0.7721207288210089.
[I 2022-08-06 07:41:27,320] Trial 2 finished with value: 0.783427243061956 and parameters: {'max_depth': 97, 'min_samples_split': 3, 'min_weight_fraction_leaf': 0.021715430024378327, 'max_features': 0.8597377465487268}. Best is trial 2 with value: 0.783427243061956.


KeyboardInterrupt: ignored

In [12]:
model = Model(train_df, "lgbm", "rgr")

In [13]:
initial_params = (
    ("n_estimators", "static", 100),
    ("objective", "static", "binary"),
    ("metric", "static", "auc"),
    ("learning_rate", "log", (1e-5, 1.0)),
    ("num_leaves", "int", (300, 2000)),
)
model.optimize(initial_params, n_trials=20)

[I 2022-08-06 07:59:08,190] A new study created in memory with name: no-name-8bb7839b-bc15-4186-92b9-42c719035901
[I 2022-08-06 07:59:20,174] Trial 0 finished with value: 0.8508964743241424 and parameters: {'learning_rate': 0.0375382101697422, 'num_leaves': 1744}. Best is trial 0 with value: 0.8508964743241424.
[I 2022-08-06 07:59:31,942] Trial 1 finished with value: 0.8251172008255213 and parameters: {'learning_rate': 0.012936354125886644, 'num_leaves': 1404}. Best is trial 0 with value: 0.8508964743241424.
[I 2022-08-06 07:59:44,056] Trial 2 finished with value: 0.7989137223114446 and parameters: {'learning_rate': 0.004247809355124376, 'num_leaves': 1996}. Best is trial 0 with value: 0.8508964743241424.
[I 2022-08-06 07:59:55,721] Trial 3 finished with value: 0.8059669513353841 and parameters: {'learning_rate': 0.005842855674677552, 'num_leaves': 1246}. Best is trial 0 with value: 0.8508964743241424.
[I 2022-08-06 08:00:06,136] Trial 4 finished with value: 0.748021939779588 and param

0.8630226961242209
{'learning_rate': 0.3225475195598088, 'num_leaves': 729}


In [22]:
optuna.visualization.plot_slice(model.optimizer.opt)

In [23]:
Evaluator(
    **model.get_model()
).run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.763676,0.781081,0.804566,0.792649,0.839295,0.291786,NaN
2,0.764927,0.765399,0.819851,0.791690,0.840958,0.295799,NaN
3,0.761175,0.774473,0.804947,0.789416,0.835163,0.299075,NaN
4,0.755549,0.786832,0.790244,0.788534,0.834108,0.295652,NaN
mean,0.761332,0.776946,0.804902,0.790572,0.837381,0.295578,1.0
